# Multi-step Logistics Scenario
This notebook wires up a shipping-focused tool stack so the agent must combine several tool calls to answer a single request.
Run the cells sequentially and finish by executing the scenario cell to watch the agent plan a shipment.

In [ ]:
import json
from typing import Annotated
from loguru import logger

from dotenv import dotenv_values

from aceai import AgentBase, ToolExecutor, LLMService, spec, tool, Tool, Graph
from aceai.llm.openai import OpenAI

values = dotenv_values(".env")

In [2]:
WAREHOUSE_ORDERS = {
    "ORD-200": {
        "destination": "Denver, CO",
        "priority": "express",
        "notes": "Helios Labs needs hardware before the Wednesday stand-up.",
        "items": [
            {"sku": "sensor-kit", "quantity": 45},
            {"sku": "control-module", "quantity": 30},
            {"sku": "battery-pack", "quantity": 60},
        ],
    },
    "ORD-207": {
        "destination": "Austin, TX",
        "priority": "standard",
        "notes": "Stocking field pod replacements.",
        "items": [
            {"sku": "telemetry-node", "quantity": 22},
            {"sku": "cooling-shroud", "quantity": 18},
        ],
    },
}

SKU_WEIGHTS = {
    "sensor-kit": 0.85,
    "control-module": 1.4,
    "battery-pack": 0.65,
    "telemetry-node": 1.1,
    "cooling-shroud": 2.2,
}

SHIPPING_RATES = {
    "standard": {"base": 48.0, "per_kg": 1.35, "eta_days": 5},
    "express": {"base": 92.0, "per_kg": 1.95, "eta_days": 2},
}

@tool
def lookup_order(
    order_id: Annotated[str, spec(description="Order identifier such as ORD-200.")]
) -> str:
    """Return line items, destination, and priority for a warehouse order."""
    order = WAREHOUSE_ORDERS.get(order_id.upper())
    if not order:
        raise ValueError(f"Unknown order {order_id}")
    return json.dumps(order)

@tool
def get_sku_weight(
    sku: Annotated[str, spec(description="Catalog SKU to pull the per-unit weight for.")]
) -> str:
    """Return the per-unit weight for a SKU in kilograms."""
    key = sku.lower()
    if key not in SKU_WEIGHTS:
        raise ValueError(f"Unknown SKU {sku}")
    return json.dumps({"sku": key, "weight_kg": SKU_WEIGHTS[key]})

@tool
def estimate_shipping_cost(
    weight_kg: Annotated[float, spec(description="Total shipment mass in kilograms.")],
    method: Annotated[str, spec(description="Shipping tier to price (standard or express).")],
) -> str:
    """Quote the shipping cost given a total weight and service tier."""
    method_key = method.strip().lower()
    if method_key not in SHIPPING_RATES:
        raise ValueError(f"Unsupported shipping method {method}")
    rates = SHIPPING_RATES[method_key]
    cost = rates["base"] + weight_kg * rates["per_kg"]
    return json.dumps(
        {
            "method": method_key,
            "weight_kg": round(weight_kg, 2),
            "cost_usd": round(cost, 2),
            "eta_days": rates["eta_days"],
        }
    )

In [3]:
import sys
import time

from aceai.llm.interface import LLMToolCall

logger.remove()
logger.add(
    sys.stdout,
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level:<7} | {message}",
    colorize=False,
)

class LoggingToolExecutor(ToolExecutor):
    async def execute_tool(self, tool_call: LLMToolCall) -> str:
        call_id = tool_call.call_id
        logger.info("Tool {name} starting (call_id={call_id}) with {arguments}", name=tool_call.name, call_id=call_id, arguments=tool_call.arguments)
        start = time.perf_counter()
        try:
            result = await super().execute_tool(tool_call)
        except Exception:
            duration = time.perf_counter() - start
            logger.exception("Tool {name} failed after {duration:.2f}s", name=tool_call.name, duration=duration)
            raise
        duration = time.perf_counter() - start
        logger.success("Tool {name} finished in {duration:.2f}s, result: {result}", name=tool_call.name, duration=duration, result=result)
        return result


In [4]:
def build_agent(prompt: str, tools: list[Tool]) -> AgentBase:
    openai_llm = OpenAI(
        api_key=values["OPENAI_API_KEY"],
        default_model="gpt-4",
        default_stream_model="gpt-4-turbo",
    )
    graph = Graph()

    llm_service = LLMService(
        providers=[openai_llm],
        timeout_seconds=120,
    )
    executor = LoggingToolExecutor(
        tools=tools,
        graph=graph,
    )

    

    return AgentBase(
        prompt=prompt,
        default_model="gpt-4",
        llm_service=llm_service,
        executor=executor,
    )

agent = build_agent(prompt=(
        "You are the logistics coordinator for AceAI. "
        "Always inspect orders, fetch SKU weights, and price shipping strictly through the available tools. "
        "Chain multiple tool calls together whenever a calculation depends on tool-derived values, "
        "and explain the final recommendation clearly."
    ), tools=[lookup_order, get_sku_weight, estimate_shipping_cost])

## Scenario: plan shipment for ORD-200
The following cell asks the agent to pull data for an urgent order, compute the shipment mass, and compare standard vs express pricing. This typically requires:
1. `lookup_order`
2. Multiple `get_sku_weight` calls (one per SKU)
3. Two `estimate_shipping_cost` calls (standard + express)

In [5]:
multi_step_question = """
You're preparing a shipping brief for Helios Labs covering order ORD-200.

Tasks:
1. Call `lookup_order` to restate the destination, priority, and every SKU with its quantity.
2. Use `get_sku_weight` for each SKU individually so you can calculate the total shipment weight in kilograms.
3. Price BOTH `standard` and `express` service levels by calling `estimate_shipping_cost` twice with the total weight.
4. Recommend which service level to book, citing cost, ETA, and the customer's stated priority.

Present the answer as a short briefing paragraph plus bullet points for weight, each quote, and the final recommendation.
"""

await agent.handle(multi_step_question)

2025-12-06 03:04:47.696 | INFO    | Tool lookup_order starting (call_id=call_w9nbZfG8nDcVCkyxazpEmDQI) with {
  "order_id": "ORD-200"
}
2025-12-06 03:04:47.697 | SUCCESS | Tool lookup_order finished in 0.00s, result: "{\"destination\": \"Denver, CO\", \"priority\": \"express\", \"notes\": \"Helios Labs needs hardware before the Wednesday stand-up.\", \"items\": [{\"sku\": \"sensor-kit\", \"quantity\": 45}, {\"sku\": \"control-module\", \"quantity\": 30}, {\"sku\": \"battery-pack\", \"quantity\": 60}]}"
2025-12-06 03:04:54.849 | INFO    | Tool get_sku_weight starting (call_id=call_VkOoNI6ci3bh8jA46pMqHY0r) with {
        "sku": "sensor-kit"
      }
2025-12-06 03:04:54.850 | SUCCESS | Tool get_sku_weight finished in 0.00s, result: "{\"sku\": \"sensor-kit\", \"weight_kg\": 0.85}"
2025-12-06 03:04:54.851 | INFO    | Tool get_sku_weight starting (call_id=call_Tjae8ooh4F6qak0NKzldhJ7Q) with {
        "sku": "control-module"
      }
2025-12-06 03:04:54.851 | SUCCESS | Tool get_sku_weight fini

"Shipping Brief for Helios Labs, Order ORD-200:\n\nThe order, destined for Denver, CO, contains 45 sensor kits, 30 control modules, and 60 battery packs. The customer prioritizes express delivery to meet their needs before the Wednesday stand-up meeting. \n\n- Calculated total shipment weight: 115.25kg\n- Standard Shipping costs $203.59 and will take approximately 5 days for delivery.\n- Express Shipping costs $316.74 and will take approximately 2 days for delivery.\n\nWhile Standard Shipping is cheaper, the customer's noted priority for quick delivery suggests that Express Shipping will best meet their needs. \n\nRecommendation:\nOpt for Express Shipping. Despite being more expensive, it aligns with the customer's preference for expeditious delivery, ensuring the hardware will arrive before the Wednesday stand-up."